In [13]:
from selenium.webdriver import Chrome
import time
import os

url = "https://www.cathaybk.com.tw/cathaybk/personal/credit-card/cards/intro/list/"

# 開啟並取得國泰信用卡網頁。
driver = Chrome("./chromedriver.exe")
# 注意是否要放大，或自訂螢幕大小。
driver.maximize_window()
time.sleep(1)
driver.get(url)

In [ ]:
#滾輪往下滾
def scrolltop(pos):
    js = "document.documentElement.scrollTop=%s" % pos
    driver.execute_script(js)
#滾輪滾到底
def scrolldown():
    js = "document.documentElement.scrollTop=10000"
    driver.execute_script(js)
#去到下一頁
def backpage(page):
    driver.find_element_by_id(
        "layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_{}".format(page-1)).click()
    # 給他三秒鐘的時間跑
    time.sleep(3)

In [ ]:
# 起始位置設為200，注意螢幕大小，我的螢幕為14吋。
page = 1
while page < 5:
    pos = 200
    for i in range(10):
        pos += 220
        scrolltop(pos)
        # 注意抓取的值為list，要一個一個取出。
        driver.find_elements_by_xpath("//div[@class='card-features']/a[@class='link-learn-more']")[i].click()
        print("現在到第幾頁:", page, "到第幾張卡片:", i + 1)
        time.sleep(1)
        if page == 1:
            driver.back()
            scrolltop(pos)
            time.sleep(1)
        else:
            driver.back()
            scrolldown()
            backpage(page)
    page += 1
    backpage(page)
print("好像沒東西喔!!!")

In [43]:
for item in range(10):
    try:
        data = driver.find_element_by_id("layout_0_rightcontent_1_tab0"+str(item+1)+"_0_divParagraph")
        print(data.text)
        print("--------------------------------------")
        driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(item + 2)).click()
    except:
        driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(item + 2)).click()
        continue

SOGO聯名卡刷LINE Pay 拿紅利，回饋率3%！
108/5/13~108/12/31 以LINE Pay 一卡通帳戶連結國泰世華帳戶，並透過LINE Pay使用已綁定的國泰世華SOGO聯名卡消費，登錄享紅利點數10 倍回饋，回饋率最高 3%！（含原卡片回饋，本活動加碼每月歸戶回饋上限 5,000 點）
  舉例說明：活動期間於 LINE Pay 一卡通連結國泰世華帳戶，且透過LINE Pay綁定SOGO聯名卡消費NT$1 萬為例，共可獲得紅利點數 5,000 點，回饋率 3%！
項目 回饋內容 紅利點數
原卡片回饋 紅利 2 倍 ( 0.6%回饋率 ) 1,000 點紅利
加碼回饋 紅利 8 倍 ( 2.4%回饋率 ) 4,000 點紅利
合計 紅利 10 倍 ( 3.0%回饋率 ) 5,000 點紅利
活動登錄

注意事項

紅利積點使用最EASY
持太平洋SOGO百貨聯名卡刷卡消費，每刷卡20元可累積1點紅利點數


注意事項

百貨店內消費9折起
持卡於全台太平洋SOGO百貨各店消費享優惠
一般商品購物享正品9折優惠
部分商品享95折優惠
超市藍標商品享9折起優惠


注意事項

太平洋SOGO百貨停車優惠
除可依現行館內購物折抵停車優惠外，再加贈優惠1小時
豐原館
其他館別：平日(週一～週四)持聯名卡可享免費停車1小時
其他館別：假日(週五～週日，含例假日)持聯名卡刷卡消費達指定門檻


注意事項

國泰人壽保費優惠
持本行信用卡刷卡繳付國泰人壽保費享以下優惠
特定壽險商品保費1%優惠
詳情以卡友權益頁面所載資訊為主
0800-036-599
https://www.cathaybk.com.tw/cathaybk/personal/credit-card/cards/rights/


注意事項
--------------------------------------
超值旅行不便險保障
持國泰世華銀行信用卡，為自已或家人（限配偶及未滿25足歲之未婚子女）支付公共運輸工具全額票款、或80%以上之旅遊團費，就可享有以下超值保險保障
保險期間：2019/1/1 12:00 至 2020/1/1 12:00 止
金卡：1500萬死傷、10萬醫療
普卡：750萬死傷、10萬醫療
※詳情以卡友權益頁面所載資訊為主


注意事項
-------------

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=77.0.3865.120)


In [21]:
# driver.find_element_by_id("layout_0_rightcontent_1_liTab03").click()

In [30]:
data = driver.find_element_by_id("layout_0_rightcontent_1_tab01_0_divParagraph")
print(data.text)

新卡滿額禮
世界卡2019/7月-2019/10月新卡刷卡禮
2019/7/1-2019/10/31期間首次申辦世界卡正卡，核卡日後60天內刷一般消費累積達門檻，享精選好禮回饋:
<滿額禮>     
精緻滿額禮：達NT$15萬以上，「Samsonite Polygon 25吋行李箱 (藍)」乙個。
尊榮滿額禮：達NT$60萬以上，「紅利積點30萬點」。
<加碼禮>     
加碼禮：達NT$18萬以上，「兩次免費機場接或送禮遇」。

贈品介紹
  精緻滿額禮：Samsonite Polygon 25吋行李箱(藍)        
材質：100%PC (聚碳酸酯)
尺寸：47 X 69 X 31 cm
淨重：4 Kg 容量：77 L
保修期間：3年 (相關保修規範以原廠官網說明為準)
商品詢問及維修電話 : 0800-088-349
商品簡介：
採用 FLAT-MAX™ 結構設計「面蓋與主箱體為 20:80 比例」，行李箱20%是拉桿系統的面蓋，80%是主要儲存空間的主箱體，打造出特大容量。
箱蓋側面有煞車止滑開關，選用日本 Hinomoto 的EASY BRAKE™煞車輪，拉桿兩側的輪子在煞車系統啟動時，仍可拖行。
TSA 海關鎖+隱藏式地址貼牌。                    
尊榮滿額禮：紅利積點30萬點
紅利積點30萬點可兌換高額航空里程/飯店積分、百貨禮券、商品，讓您輕鬆旅行、購物，盡享寫意自在品味生活。(瞭解更多「舞動紅利」紅利點數兌換辦法)
  紅利積點30萬點可兌換多樣化優惠：
亞洲萬里通74,000里數，或長榮航空60,000哩，或Le Club AccorHotels雅高樂雅會獎勵積分20,000點
新光三越百貨商品禮券17,000元，或遠東SOGO百貨商品券17,500元
TreeMall紅利線上購物全站超過10萬種品項
加碼禮：兩次免費機場接或送禮遇
享指定區域至以下機場免費機場接或送禮遇2次。
台北松山機場
桃園國際機場
台中國際機場
高雄國際機場(小港機場)
24小時服務預約專線 (04) 2206-1608
(持卡人倘符合消費資格，本行將透過簡訊通知，持卡人收到簡訊後才可進行預約。)
(本免費機場接送禮遇有限定服務區域，詳細服務適用條件與限制依官網公告為準。)


注意事項









生日貴賓尊寵禮遇
Sisley
生日